# 量子态直接保真度估计

*版权所有 (c) 2022 百度量子计算研究所，保留所有权利。*

## 内容概要

量子态层析技术能够给我们提供量子态的完整信息，但是它十分消耗资源，并且大部分情况我们想要的信息只是实际制备量子态的保真度。直接保真度估计（Direct Fidelity Estimation，DFE）是一种高效获得保真度的方法。

本教程将介绍直接保真度估计的理论及其在[百度量子平台](https://quantum.baidu.com/)上的实践。

## 理论


### 保真度估计

我们将想要制备的纯态（pure state）记为 $\rho$ ，将我们实际制备的量子态记为 $\sigma$ ，那么保真度被定义为如下式子，

$$
    \mathcal{F}(\rho, \sigma) = \textrm{Tr}[\rho\sigma].
$$

我们用 $P_k(k=1,\cdots,d^2)$ 来表示所有的 Pauli 算符，其中 $d=2^n$ ， $n$ 是量子比特数。量子态 $\sigma$ 和 $\rho$ 都可以在这一组 Pauli 算符下展开。我们定义一个函数 $\chi_\rho(k) = \textrm{Tr}[\rho P_k/\sqrt{d}]$ ，那么 $\rho$ 的保真度可以写为如下形式，

$$
\begin{align*}
        \textrm{Tr}[\rho\sigma]
    &=  \langle\langle \rho | \sigma \rangle\rangle \\
    &=  \sum_k \langle\langle \rho | k \rangle\rangle \langle\langle k | \sigma \rangle\rangle \\
    &=  \sum_k \textrm{Tr}[\rho P_k/\sqrt{d}]\textrm{Tr}[\sigma P_k/\sqrt{d}] \\
    &=  \sum_k \chi_\rho(k)\chi_\sigma(k).
\end{align*}
$$

下面，我们将介绍如何构建一个估计器，用于估计保真度 $\textrm{Tr}[\rho\sigma]$ 。首先，我们对 $k\in\left\{1, \cdots, d^2\right\}$ 按照如下概率进行随机采样，

$$
    \textrm{Pr}(k) = \chi^2_\rho(k).
$$

对于每个被采样到的 $k$ ，我们将利用 $P_k$ Pauli 测量来估计 $\chi_\sigma(k)$ 值。估计器 $X$ 被定义为如下式子，

$$
    X = \chi_\sigma(k) / \chi_\rho(k).
$$

很显然，$\mathbb{E}X = \sum_k\textrm{Pr}(k)X = \textrm{Tr}[\rho\sigma]$ ，也就是说 $X$ 是对保真度 $\textrm{Tr}[\rho\sigma]$ 的无偏估计。


### 参数设置

还有两个问题需要我们去处理：

1. 我们需要进行多少次测量？
2. 对于每个 Pauli 测量，我们需要重复多少次测量操作？

下面我们将解决这两个问题。首先，我们定义两个变量，给定误差 $\epsilon$ 和错误率 $\delta$ 。利用 Chebyshev's 不等式和 Hoeffding's 不等式，我们便可以计算获得采样次数 $\ell$ 和每次测量的操作次数 $m_i$ [1]。

我们可以做如下总结，真实保真度 $\mathcal{F}(\rho, \sigma) \in [\tilde{\mathcal{F}}(\rho, \sigma)-2\epsilon, \tilde{\mathcal{F}}(\rho, \sigma)+2\epsilon]$ 的概率 $\geq 1-2\delta$ ，其中 $\tilde{\mathcal{F}}(\rho, \sigma)$ 是我们估计的保真度。


## 实践

下面，我们将演示如何将直接保真度测量应用在贝尔态上。

首先，我们导入一些必要的包。

In [ ]:
import QCompute
import qcompute_qep.estimation as estimation

然后，我们在第二个和第三个量子比特上，搭建制备贝尔态的量子电路。

In [ ]:
qp = QCompute.QEnv()  # qp is short for "quantum program", instance of QProgram
qp.Q.createList(3)
QCompute.H(qp.Q[1])
QCompute.CX(qp.Q[1], qp.Q[2])

![DFE](./figures/dfe-bell-state.png "图1: 贝尔态制备电路。 ")

现在，我们成功搭建如上电路。

接下来我们需要选择量子计算机的类型，可以选择模拟器或量子计算机真机，这里我们选择本地模拟器。剩下的步骤就很简单了，只需要调用我们的 DFEState 模块则可完成直接保真度估计。这里，我们设置参数 $\epsilon=\delta=0.05$ ，并且将目标比特设置为第二个和第三个量子比特（`qubits=[1, 2]`）。

In [ ]:
# 使用本地模拟器
qc = QCompute.BackendName.LocalBaiduSim2

# 请登录“量易伏”平台（https://quantum-hub.baidu.com/）获取 Token
# QCompute.Define.hubToken = "Token"
# qc = QCompute.BackendName.CloudBaiduQPUQian

# Initialize a DFEState instance
est = estimation.DFEState()

# Call the estimate procedure
est.estimate(qp, qc, qubits=[1, 2], epsilon=0.05, delta=0.05)

最后，我们可以计算得到贝尔态的保真度。

In [ ]:
print('Fidelity between the ideal and noisy states is: F = {:.5f}'.format(est.fidelity))

## 参考资料

[1] Flammia, Steven T., and Yi-Kai Liu. "Direct fidelity estimation from few Pauli measurements." **[Physical Review Letters](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.106.230501)** 106.23 (2011): 230501.

[2] da Silva, Marcus P., Olivier Landon-Cardinal, and David Poulin. "Practical characterization of quantum devices without tomography." **[Physical Review Letters](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.107.210404)** 107.21 (2011): 210404.

[3] Kliesch, Martin, and Ingo Roth. "Theory of Quantum System Certification." **[PRX Quantum](https://journals.aps.org/prxquantum/abstract/10.1103/PRXQuantum.2.010201)** 2.1 (2021): 010201.